In [1]:
# Preprocessing the extrapolated COMPAS dataset

import csv
import random
import pandas as pd
from hypothesis_test import *
from kardis_test import *
from mg_calculator import *
from q_finder import *

file = open(r"Datasets/compas.csv", mode='r', encoding='utf-8-sig')
csvreader = csv.reader(file)
rowlist = [row for row in csvreader]
for category in rowlist[0]:
    index = rowlist[0].index(category)
    exec(category + " = [row[index] for row in rowlist[1:]]")
file.close()

ids = [int(id[x]) for x in range(len(id))] 
person_id = [int(person_id[x]) for x in range(len(person_id))]

file = open(r"Datasets/people.csv", mode='r', encoding='utf-8-sig')
csvreader = csv.reader(file)
rowlist = [row for row in csvreader]
for category in rowlist[0]:
    index = rowlist[0].index(category)
    exec(category + " = [row[index] for row in rowlist[1:]]")
file.close()

idx = 0
people_id = [int(id[x]) for x in range(len(id))] 
races = []
sexes = []
for x in person_id:
    if x == people_id[idx]:
        races.append(race[people_id[idx] - 1])
        sexes.append(race[people_id[idx] - 1])
    else:
        idx += 1

new_compas = list(filter(lambda x: not x[1] == 'N/A', list(zip(sex, races))))
compas_extrapolated_100k = []

for x in range(100000):
    idx = random.randrange(len(new_compas))
    compas_extrapolated_100k.append(new_compas[idx])

if len(compas_extrapolated_100k) == 100000 and type(compas_extrapolated_100k) == list:
    print('Data processed successfully.')
else:
    print('Data processing failed.')

Data processed successfully.


In [2]:
def data_bin_slicer_multival(nbdata, nbhypothesis, nbvaluelist, selectedvaluelist):
    """
    This function takes in a list of data with nbvaluelist possible values
    and combines values to reduce it down to len(selectedvaluelist) values. 
    It is used to control the number of possible values for our runtime experiments.
    
    ex. nbvaluelist = [1,2,3,4]
    selectedvaluelist = [1,2]
    nvaluelist = [[1],[2],["not-([1,2])"]]
    """
    nvaluelist = []
    nhypothesis = []
    ndata = []
    biasval_name = '('+ str(selectedvaluelist[0])
    for val in selectedvaluelist[1:]:
        biasval_name += ", " + str(val)
    biasval_name+=')'
    
    for val in range(len(selectedvaluelist)):
        nvaluelist.append(selectedvaluelist[val])
    
    nvaluelist.append(['not-'+biasval_name])
    num_biasval = [nbdata.count(x) for x in selectedvaluelist]
    ncounts = num_biasval
    
    ncounts.append(int(len(nbdata))-int(sum(num_biasval)))
    for x in range(len(selectedvaluelist)):
        ndata += [selectedvaluelist[x]]*num_biasval[x]
    
    ndata += [['not-'+biasval_name]]*ncounts[-1]
            
    for y in range(len(selectedvaluelist)):
        nhypothesis.append(nbhypothesis[nbvaluelist.index(selectedvaluelist[y])])
        
    sum_nhyp = 0
    
    for s in range(len(nhypothesis)):
        sum_nhyp += nhypothesis[s]
    nhypothesis.append(1-sum_nhyp)
    return (ndata,ncounts,nhypothesis,nvaluelist)

In [3]:
# Setup and Experiment

value_list = [['Male','Caucasian'],['Female','Caucasian'],['Male','African-American'],['Female','African-American'],
              ['Male','Hispanic'],['Female','Hispanic'],['Male','Asian'],['Female','Asian'],
              ['Male','Other'],['Female','Other']]
hypothesis = 10*[1/10] #uniform hypothesis
alpha = 0.05 
data = compas_extrapolated_100k
num_values = 1

while num_values < 10:
    selectedvalue = []
    for idx in range(num_values):
        selectedvalue += [value_list[idx]]
    ndata, ncounts, nhypothesis, nvaluelist = data_bin_slicer_multival(data, hypothesis, value_list, selectedvalue)
    my_time = %timeit -n 1 -r 10 -o hypothesis_test_silent(ndata, nvaluelist, alpha, nhypothesis)
    num_values += 1

1.68 s ± 14 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
1.88 s ± 23.2 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
2.43 s ± 19.8 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
3.91 s ± 24.4 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
7.66 s ± 60.5 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
15.4 s ± 136 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
22.4 s ± 246 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
17.4 s ± 163 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
1.63 s ± 30.5 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
